In [8]:
# src/03_predict_footer_msg.py
from ultralytics import YOLO
from pathlib import Path
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import textwrap

MODEL_WEIGHTS = "../outputs/food7_yolov8n/weights/best.pt"
SOURCE        = "../data/valid/images"   # image path or folder
OUT_DIR       = Path("outputs/food7_footer")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Messages WITHOUT emoji to avoid "?" issues in OpenCV/PIL default fonts ---
FOOD_INFO = {
    "pizza":       {"kcal": 285, "protein": 12,  "unit": "per slice",  "score": "C",
                    "advice": "Tasty but calorie-dense. Add lean protein or balance with salad."},
    "burger":      {"kcal": 800, "protein": 45,  "unit": "per burger", "score": "B",
                    "advice": "Strong protein, very high calories. Choose lean meat or smaller portion."},
    "fries":       {"kcal": 312, "protein": 8,   "unit": "per serving","score": "D",
                    "advice": "Low protein for the calories. Keep portions small; pair with protein."},
    "fried-egg":   {"kcal": 72,  "protein": 6.2, "unit": "per egg",    "score": "A",
                    "advice": "Excellent protein-to-calorie ratio. 2–3 eggs make a strong base."},
    "cereal":      {"kcal": 300, "protein": 6,   "unit": "per serving","score": "D",
                    "advice": "High calories, low protein. Add milk or Greek yogurt for protein."},
    "salad":       {"kcal": 180, "protein": 4,   "unit": "per serving","score": "C",
                    "advice": "Great low-calorie filler. Add chicken, beans, or eggs for protein."},
    "steak":       {"kcal": 500, "protein": 45,  "unit": "per serving","score": "A",
                    "advice": "Excellent protein source. Pair with vegetables for balance."},
}

# Try to load a TrueType font. If you lack this file, fall back to default.
def get_font(size=24):
    for fp in [
        "C:/Windows/Fonts/arial.ttf",            # Windows
        "/System/Library/Fonts/Supplemental/Arial.ttf",  # macOS (may vary)
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", # Linux
    ]:
        try:
            return ImageFont.truetype(fp, size=size)
        except Exception:
            continue
    return ImageFont.load_default()

FONT = get_font(24)
SMALL = get_font(22)

def normalize_label(lbl: str) -> str:
    """Normalize class label to match FOOD_INFO keys."""
    return lbl.strip().lower()

def combined_message(detected_labels):
    """Build one combined message for the image: per-class line + overall tips."""
    if not detected_labels:
        return "No foods detected."

    lines = []
    uniq = []
    for x in detected_labels:
        if x not in uniq:
            uniq.append(x)

    for lbl in uniq:
        key = normalize_label(lbl)
        info = FOOD_INFO.get(key)
        if info:
            lines.append(
                f"{lbl}: ~{info['kcal']} kcal / {info['protein']} g protein {info['unit']} — Score {info['score']}"
            )
        else:
            lines.append(f"{lbl}: add your calories/protein info — Score ?")

    scores = [FOOD_INFO.get(normalize_label(x), {}).get("score", "?") for x in uniq]
    if "A" in scores or "B" in scores:
        if "D" in scores or "E" in scores:
            lines.append("Tip: Combine high-protein items (A/B) with lower-calorie sides; limit D/E items.")
        else:
            lines.append("Tip: Good protein density detected. Keep portions reasonable.")
    else:
        lines.append("Tip: Protein density is low. Add lean protein (eggs, chicken, beans).")

    for lbl in uniq:
        key = normalize_label(lbl)
        info = FOOD_INFO.get(key)
        if info:
            lines.append(f"{lbl} advice: {info['advice']}")

    return "\n".join(lines)

def draw_footer_box_below(image_bgr: np.ndarray, text: str,
                          max_width_ratio=0.95, padding=12, line_spacing=6,
                          box_fill=(0, 0, 0, 180), text_fill=(255, 255, 255, 255)):
    """Return a new BGR image with an opaque/semi-transparent footer box containing wrapped text."""
    h, w = image_bgr.shape[:2]
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(image_rgb)

    draw = ImageDraw.Draw(pil_img)
    max_text_width = int(w * max_width_ratio)

    wrapped_lines = []
    for para in text.split("\n"):
        trial = textwrap.wrap(para, width=80) if para else [""]
        for line in trial:
            if not line:
                wrapped_lines.append("")
                continue
            while True:
                tw, th = draw.textbbox((0, 0), line, font=SMALL)[2:]
                if tw <= max_text_width or len(line) <= 5:
                    break
                line = line[:-5] + "..."
            wrapped_lines.append(line)

    line_heights = []
    for line in wrapped_lines:
        bbox = draw.textbbox((0, 0), line, font=SMALL)
        lh = bbox[3] - bbox[1]
        line_heights.append(lh)
    text_height = sum(line_heights) + (len(line_heights) - 1) * line_spacing
    footer_h = text_height + 2 * padding

    new_h = h + footer_h
    new_img = Image.new("RGBA", (w, new_h), (0, 0, 0, 0))
    new_img.paste(pil_img, (0, 0))

    footer = Image.new("RGBA", (w, footer_h), box_fill)
    new_img.alpha_composite(footer, (0, h))

    draw2 = ImageDraw.Draw(new_img)
    x = int((w - max_text_width) / 2)
    y = h + padding
    for line, lh in zip(wrapped_lines, line_heights):
        draw2.text((x, y), line, font=SMALL, fill=text_fill)
        y += lh + line_spacing

    final_rgb = new_img.convert("RGB")
    final_bgr = cv2.cvtColor(np.array(final_rgb), cv2.COLOR_RGB2BGR)
    return final_bgr

def process_image(img_path: Path, model: YOLO):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Could not read: {img_path}")
        return

    r = model.predict(source=str(img_path), conf=0.35, iou=0.5, verbose=False, device=0)[0]
    names = r.names

    labels = []
    if r.boxes is not None and len(r.boxes) > 0:
        for b in r.boxes:
            cls_id = int(b.cls.item())
            labels.append(names[cls_id])

    msg = combined_message(labels)
    out_img = draw_footer_box_below(img, msg)

    out_path = OUT_DIR / img_path.name
    cv2.imwrite(str(out_path), out_img)
    print(f"Saved: {out_path}")

if __name__ == "__main__":
    model = YOLO(MODEL_WEIGHTS)

    src = Path(SOURCE)
    if src.is_dir():
        for p in sorted(src.glob("*.*")):
            process_image(p, model)
    else:
        process_image(src, model)

    print(f"\nDone. Check: {OUT_DIR.resolve()}")


Saved: outputs\food7_footer\240_F_417468972_FlB459aHL00FmW0pUxSihoxkYHb9Zyge_jpg.rf.7afc212200f090bebba8ff37ef7846ef.jpg
Saved: outputs\food7_footer\240_F_53231871_PGKkG6Tmfyy4x41k17KNnw2QFDaNGsyW_jpg.rf.d1883c1c44aae35bd9fb84ee698b0b7f.jpg
Saved: outputs\food7_footer\26_jpg.rf.b94d6c396a1cc1df6fb044ecd8dd6645.jpg
Saved: outputs\food7_footer\39_jpg.rf.60b03921c7af025885ffcd22ddc44258.jpg
Saved: outputs\food7_footer\7_jpg.rf.aa594cd6b8e1c4fe28529ba9eab4ecba.jpg
Saved: outputs\food7_footer\appetizing-burger-with-double-meat-delicious-burger-with-french-fries-kemt3e_jpg.rf.5df704bc064adb66c74a1dfa6f18d587.jpg
Saved: outputs\food7_footer\bacon-burger-on-wooden-plate-MKGJ1D_jpg.rf.8e0708f77f785d0149deb9b802265ad5.jpg
Saved: outputs\food7_footer\daf1d1b8-8de1-41d9-b74d-b7457fb63548_jpeg.rf.ef3730fc1d980cf4b8f375267f498121.jpg
Saved: outputs\food7_footer\delicious-burger-and-french-fries-with-two-kinds-of-sauce-2jmk6p5_jpg.rf.21373c48fd4950c768082cbaaa1a42f3.jpg
Saved: outputs\food7_footer\fa